<a href="https://colab.research.google.com/github/Sandeep10021/DocumentClassifier/blob/main/DocumentClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install keras-tuner

In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
import numpy as np
from PIL import Image

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
path = '/content/gdrive/MyDrive/ML and DL Datasets/Document Classifier'

In [4]:
print(os.listdir('/content/gdrive/MyDrive/ML and DL Datasets/Document Classifier'))

['AadharCard', 'VoterID', 'DrivingLicence', 'EmiratesID', 'PanCard']


In [18]:
images=[]
labels=[]
folders = os.listdir(path)
for i in range(len(folders)):
    for ele in os.listdir(path+'//'+folders[i]):
        img = Image.open(path+'//'+folders[i]+'//'+ele)
        img = img.resize((256,256))
        img = img.convert('L')
        images.append((np.array(img))/255.0)
        labels.append(i)

In [19]:
print(len(images))
print(len(labels))

139
139


In [35]:
images = np.array(images)
labels = np.array(labels)
images = images.reshape(len(images),256,256,1)

In [36]:
print(images[0].shape)

(256, 256, 1)


In [37]:
train_images= images
train_labels = labels

In [40]:
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32,max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
            activation='relu',
            input_shape=(256,256,1)),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32,max_value=128, step=16),
            kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
            activation='relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units',min_value=32, max_value=128,step=16),
            activation = 'relu'),
        keras.layers.Dense(10, activation='softmax')   
    ])

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

    return model

In [41]:
tuner_search = RandomSearch(build_model, objective="val_accuracy", max_trials=5, directory='Output',project_name="Document Classifier")
tuner_search.search(train_images,train_labels,epochs=2, validation_split=0.1)

Trial 5 Complete [00h 03m 22s]
val_accuracy: 0.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 10m 52s
INFO:tensorflow:Oracle triggered exit


In [42]:
model = tuner_search.get_best_models(num_models=1)[0]

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 252, 252, 112)     2912      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 248, 248, 96)      268896    
_________________________________________________________________
flatten (Flatten)            (None, 5904384)           0         
_________________________________________________________________
dense (Dense)                (None, 80)                472350800 
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 472,623,418
Trainable params: 472,623,418
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=2)

Epoch 3/10
4/4 [==============================] - 3s 707ms/step - loss: 2.6084 - accuracy: 0.3280 - val_loss: 10.3370 - val_accuracy: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 3s 652ms/step - loss: 3.5202 - accuracy: 0.3520 - val_loss: 2.2361 - val_accuracy: 0.0714
Epoch 5/10
4/4 [==============================] - 3s 650ms/step - loss: 2.6573 - accuracy: 0.4480 - val_loss: 2.1185 - val_accuracy: 0.2857
Epoch 6/10
4/4 [==============================] - 3s 658ms/step - loss: 2.2496 - accuracy: 0.4640 - val_loss: 0.2180 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 3s 659ms/step - loss: 2.6201 - accuracy: 0.4720 - val_loss: 2.5762 - val_accuracy: 0.0714
Epoch 8/10
4/4 [==============================] - 3s 675ms/step - loss: 0.7374 - accuracy: 0.7840 - val_loss: 1.8527 - val_accuracy: 0.2143
Epoch 9/10
4/4 [==============================] - 3s 654ms/step - loss: 0.5224 - accuracy: 0.7920 - val_loss: 1.3374 - val_accuracy: 0.2857
Epoch 10/10
4/4

In [45]:
# Save the model:
model.save(path)

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/ML and DL Datasets/Document Classifier/assets
